# Homework 4
###### by Mher movsisyan
---

### Problem 1 (10 point):  
The Poisson distribution may be useful to model events such as  
a) Time interval between train arrivals  
b) The price of the house  
c) The time of the first bus arriving to a bus station between 12:00 and 16:00  
d) The score of soccer matches.  

Answer: d

### Problem 2 (40 points)  
The data `graduate.csv` has a binary response (outcome, dependent) variable called admit, which is student’s result of getting into graduate school. There are three predictor variables: gre, gpa and rank. We will treat the variables gre and gpa as continuous. The variable rank takes on the values 1 through 4. Institutions with a rank of 1 have the highest prestige, while those with a rank of 4 have the lowest.  

a) Construct logistic regression model with dependent variable admit and independent variables gre, gpa and rank. Explain coefficients and show which coefficients are significant. (10 points)  

In [64]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.tools import add_constant

(df := pd.read_csv("graduate.csv"))

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4
...,...,...,...,...
395,0,620,4.00,2
396,0,560,3.04,3
397,0,460,2.63,2
398,0,700,3.65,2


In [65]:
x = add_constant(df[["gre", "gpa", "rank"]])
y = df[["admit"]]

logreg = sm.Logit(y, x).fit()
logreg.summary()

Optimization terminated successfully.
         Current function value: 0.574302
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  400
Model:                          Logit   Df Residuals:                      396
Method:                           MLE   Df Model:                            3
Date:                Sun, 16 Oct 2022   Pseudo R-squ.:                 0.08107
Time:                        19:04:28   Log-Likelihood:                -229.72
converged:                       True   LL-Null:                       -249.99
Covariance Type:            nonrobust   LLR p-value:                 8.207e-09
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.4495      1.133     -3.045      0.002      -5.670      -1.229
gre            0.0023      0.001      2.101      0.036       0.000       0.004
gpa            0.7770      0.327      2.373      0.018       0.135       1.419
rank          -0.5600      0.127     -4.405      0.000      -0.809      -0.311
==============================================================================
"""

At `alpha = 0.05`, all coefficients are significant. As expected, rank has negative effect on the estimate, while gre and gpa are positive.

b) Find the minimum and maximum values of predictions and discuss how well the
model predicts the probability of admit.  

In [66]:
logreg.predict(x).sort_values()

289    0.048792
48     0.059898
71     0.061077
83     0.071967
304    0.081210
         ...   
69     0.673454
150    0.675160
12     0.698862
293    0.713068
2      0.717814
Length: 400, dtype: float64

We see that the lowest value is 0.049 and the highest is 0.718. This means that the model is not sure in the positive predictions.

c) What is the probability for students with median GPA and GRE from 3-th rank
institution to admit into graduate school? (10 points)

In [67]:
logreg.predict([1, df.gre.median(), df.gpa.median(), 3])[0]

0.23845203790949082

Approximatelly 23.8%

d) What is odd ratio for students with maximum GPA and maximum GRE from 4-th
rank institution? (10 points)

In [68]:
max_pred_prob = logreg.predict([1, df.gre.max(), df.gpa.max(), 4])[0]
(odds := max_pred_prob/(1-max_pred_prob))

0.47404546382958435

The odds ratio is 0.474

### Problem 3 (50 points)  
In this data `students.csv`, num_awards is the outcome variable and indicates the number of
awards earned by students at a high school in a year. math is a continuous predictor variable
and represents students’ scores on their math final exam, and prog is a categorical predictor
variable with three levels indicating the type of program in which the students were
enrolled. It is coded as 1 = “General”, 2 = “Academic” and 3 = “Vocational”  
  
a) Estimate a Poisson regression model for `num_awards`, including `math` and `prog`ram
(as multiple dummy variable) as explanatory variables. Interpret coefficients
and the significance of the coefficients (10 points)

In [69]:
(df := pd.read_csv("students.csv"))

,id,num_awards,prog,math
0,45,0,3,41
1,108,0,1,41
2,15,0,3,44
3,67,0,3,42
4,153,0,3,40
...,...,...,...,...
195,100,2,2,71
196,143,2,3,75
197,68,1,2,71
198,57,0,2,72


In [70]:
from statsmodels.genmod.families import Poisson
from sklearn.preprocessing import OneHotEncoder
import numpy as np

ohe = OneHotEncoder(sparse=False, drop="first")
encoded_prog = ohe.fit_transform(df[["prog"]])

x = pd.concat([df[["math"]], pd.DataFrame(encoded_prog, columns=["academic", "vocational"])], axis=1)
x = add_constant(x)
y = df[["num_awards"]]

(pois_model := sm.GLM(y, x, family = Poisson()).fit()).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:             num_awards   No. Observations:                  200
Model:                            GLM   Df Residuals:                      196
Model Family:                 Poisson   Df Model:                            3
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -182.75
Date:                Sun, 16 Oct 2022   Deviance:                       189.45
Time:                        19:05:25   Pearson chi2:                     212.
No. Iterations:                     5   Pseudo R-squ. (CS):             0.3881
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.2471      0.658     -7.969      0.000      -6.538      -3.957
math           0.0702      0.011      6.619      0.000       0.049       0.091
academic       1.0839      0.358      3.025      0.002       0.382       1.786
vocational     0.3698      0.441      0.838      0.402      -0.495       1.234
==============================================================================
"""

All variables of the model are significant at `alpha = 0.05` except for `vocational` (`general` taken as a reference point). All variables contribute to the prediction in a positive way.

b) Obtain predictions from the Poisson regression model. Find the minimum and
maximum values and discuss how well the model predicts number of awards.
(10 points)

In [71]:
pois_model.predict(x).sort_values()

104    0.061311
8      0.077128
15     0.081172
47     0.081172
1      0.093399
         ...   
198    2.429553
178    2.429553
172    2.429553
199    2.606113
189    2.998657
Length: 200, dtype: float64

In [72]:
((pois_model.predict(x) > np.array(y).flatten() - 0.5) & (pois_model.predict(x) < np.array(y).flatten() + 0.5)).sum()/200

0.625

I wouldn't consider this model a good predictive model. However, the minimum predicted value is close to 0 and the max is close to 3 which is good

c) Find the expected number of awards of students who has 52 math score and is
from program 2. (7 points)


In [73]:
pois_model.predict([1, 52, 1, 0])[0]

0.5972971305773277

0.597 awards

d) Find the expected number of awards of students who has 50 math score and is
from program 3. (8 points)

In [75]:
pois_model.predict([1, 50, 0, 1])[0]

0.2541843992631556

0.254 expected number of awards

e) Calculate the probability that a students from c) and d) together will get at least
one award. (Considering they are independent from each other. (15 points)

In [77]:
from scipy.stats import poisson

p_lambda = pois_model.predict([1, 50, 0, 1])[0] + pois_model.predict([1, 52, 1, 0])[0]

poisson.cdf(1, mu=p_lambda)

0.7901793102308375

The probability is 79%